In [63]:
%load_ext autoreload
%autoreload 2
from transformers import BertModel, BertConfig
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import datasets
import numpy as np
from datasets import load_from_disk
import pickle as pkl
import sklearn
from spacy.lang.en import English
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegressionCV
from collections import defaultdict
from copy import deepcopy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loosely following [this tutorial](https://amueller.github.io/aml/05-advanced-topics/13-text-data.html).

In [59]:
dataset = datasets.load_dataset('sst2')
nlp = English()
simple_tokenizer = lambda x: [str(x) for x in nlp.tokenizer(x)] # for our word-finding

Using custom data configuration default
Reusing dataset sst2 (/tmp/.xdg_cache_vision/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
def transform(vectorizer, dataset):
    X_train = vectorizer.transform(dataset['train']['sentence'])
    X_val = vectorizer.transform(dataset['validation']['sentence'])
    X_test = vectorizer.transform(dataset['test']['sentence'])
    return X_train, X_val, X_test


r = defaultdict(list)
for i, vectorizer in enumerate([
    CountVectorizer(tokenizer=simple_tokenizer, ngram_range=(1, 2)),
    TfidfVectorizer(tokenizer=simple_tokenizer, ngram_range=(1, 2)),
    CountVectorizer(tokenizer=simple_tokenizer), # unigrams only
    TfidfVectorizer(tokenizer=simple_tokenizer)
]):
    print(vectorizer)
    vectorizer.fit(dataset['train']['sentence'])
    X_train, X_val, X_test = transform(vectorizer, dataset)
    print('\tshapes', X_train.shape, X_val.shape)
    print('\tvocab ex.', list(vectorizer.vocabulary_.keys())[:10])
    r['vectorizer'].append(deepcopy(vectorizer))
    r['vectorizer_type'].append(str(type(vectorizer)))
    r['ngram_range'].append(vectorizer.ngram_range)
    
    
    # model
    m = LogisticRegressionCV()
    m.fit(X_train, dataset['train']['label'])
    r['model'].append(deepcopy(m))
    
    # performance
    print('\ttrain', m.score(X_train, dataset['train']['label']).round(3))
    print('\tval', m.score(X_val, dataset['validation']['label']).round(3),
          np.mean(dataset['validation']['label']).round(3))
    r['acc_train'].append(m.score(X_train, dataset['train']['label']))
    r['acc_val'].append(m.score(X_train, dataset['validation']['label']))
    
    pkl.dump(r, open(f'r_{i}.pkl', 'wb'))

CountVectorizer(ngram_range=(1, 2),
                tokenizer=<function <lambda> at 0x7eff445d7b80>)
	shapes (67349, 86388) (872, 86388)
	vocab ex. ['hide', 'new', 'secretions', 'from', 'the', 'parental', 'units', 'hide new', 'new secretions', 'secretions from']


/accounts/projects/vision/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/accounts/projects/vision/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mode

In [66]:
str(type(vectorizer))

"<class 'sklearn.feature_extraction.text.TfidfVectorizer'>"

In [65]:
vectorizer.ngram_range

(1, 1)